# Constraints

Constraints on binary variables are defined in a similar way to [objective funtions](./objective_functions.ipynb) and we recommend reading about objective functions first.

#### Basic Example
Consider 3 variables $a, b$, and $c$ which take on values 0 and 1. Suppose that we want to add a constraint that $a$ and $b$ cannot both be zero. Since $a, b \in \{0, 1\}$,
this constraint holds if and only if
$$
a + b > 0
$$

To specify this constraint on Forge, we build the polynomial $g(a, b, c) = a + b$ and we say that $g$ is positive.

In [1]:
# See objective_functions.ipynb to understand how to define g
g = {(0,): 1, (1,): 1}

constraint_dict = {'POSITIVE': [g]}

Now we encapsulate this dictionary into Forge's Constraints type:

In [2]:
from qcware.types.optimization import Constraints

cnstr = Constraints(
    constraint_dict,
    num_variables=3,
)

print(cnstr)

Constraints:
    Number of variables: 3
    Total number of constraints: 1
    Variable domain: boolean

    Predicate      Number of Constraints    Highest Degree
    -----------  -----------------------  ----------------
    POSITIVE                           1                 1

    x_0 + x_1 > 0


#### Multiple constraints

Suppose we also demand that $b + c > 0$. We now have two functions that must be positive:
- $a + b > 0$
- $b + c > 0$

Defining this is easy on Forge:

In [3]:
# Constrained polynomials:
g = {(0,): 1, (1,): 1}  # g(a, b, c) = a + b
h = {(1,): 1, (2,): 1}  # h(a, b, c) = b + c

# dict defining constraints:
constraints = {
    'POSITIVE': [g, h]
}

# Constraints object:
constraints = Constraints(
    constraints,
    num_variables=3
)
print(constraints)

Constraints:
    Number of variables: 3
    Total number of constraints: 2
    Variable domain: boolean

    Predicate      Number of Constraints    Highest Degree
    -----------  -----------------------  ----------------
    POSITIVE                           2                 1

    x_0 + x_1 > 0
    x_1 + x_2 > 0


#### Types of constraints

##### Predicates

There are six possible conditions that we can directly put on a polynomial $g$:


<table style="width:60%; text-align:center;">
  <tr>
    <th>Constraint</th>
    <th>Code</th>
  </tr>
  <tr>
    <td> $g = 0$ </td>
    <td> <code>{'ZERO': [g]}</code> </td>
  </tr>
  <tr>
      <td>$g \neq 0$</td>
      <td><code>{'NONZERO': [g]}</code></td>
  </tr>
  <tr> 
    <td>$g > 0$</td>
    <td><code>{'POSITIVE': [g]}</code></td>
  </tr>
  <tr>
    <td>$g < 0$</td>
    <td><code>{'NEGATIVE': [g]}</code></td>
  </tr>
  <tr>
    <td>$g \geq 0$</td>
    <td><code>{'NONNEGATIVE': [g]}</code></td>
  </tr>
  <tr>
    <td>$g \leq 0$</td>
    <td><code>{'NONPOSITIVE': [g]}</code></td>
  </tr>
</table>

Rather than typing `'NONPOSITIVE'`, etc., using `Predicate` is good practice as it gives access to autocompletion and catches typos:

In [4]:
from qcware.types.optimization import Predicate

# No need to worry about mistyping when using Predicate
constraints = {
    Predicate.POSITIVE: [g],
    Predicate.NONNEGATIVE: [h]
}

##### Nonlinear constraints

Nonlinear constraints like $a b c = 0$ are constructed just like other [polynomials](./objective_functions.ipynb) are.

In [5]:
constraints = {
    Predicate.ZERO: [
        {(0, 1, 2): 1}  # a * b * c = 0
    ]
}

constraints = Constraints(
    constraints,
    num_variables=3
)
print(constraints)

Constraints:
    Number of variables: 3
    Total number of constraints: 1
    Variable domain: boolean

    Predicate      Number of Constraints    Highest Degree
    -----------  -----------------------  ----------------
    ZERO                               1                 3

    x_0 x_1 x_2 = 0


#### Problem instances

Problem instances consist of:
1. An [objective function](./objective_functions.ipynb)
2. Optional constraints

These instances are the inputs for quantum optimization algorithms on Forge.

In [6]:
from qcware.types.optimization import BinaryProblem, PolynomialObjective

# Build objective function
objective_polynomial = {
    (): 4,
    (0,): 1, 
    (1,): 1, 
    (2,): -1, 
    (0, 1): 3,
    (1, 2): 1, 
}

objective_function = PolynomialObjective(
    polynomial=objective_polynomial, 
    num_variables=3,
)

constraints = {
    Predicate.POSITIVE: [
        {(0,): 1, (1,): 1},  # a + b > 0
        {(1,): 1, (2,): 1},  # b + c > 0
    ],
    Predicate.ZERO: [
        {(0, 1, 2): 1} # a * b * c = 0
    ]
}

constraints = Constraints(
    constraints,
    num_variables=3
)


# Construct problem instance
problem_instance = BinaryProblem(
    objective=objective_function,
    constraints=constraints
)

print(problem_instance)

Objective:
    4 + x_0 + x_1 - x_2 + 3 x_0 x_1 + x_1 x_2  (3 boolean variables)

Constraints:
    Number of variables: 3
    Total number of constraints: 3
    Variable domain: boolean

    Predicate      Number of Constraints    Highest Degree
    -----------  -----------------------  ----------------
    ZERO                               1                 3
    POSITIVE                           2                 1

    x_0 x_1 x_2 = 0


    x_0 + x_1 > 0
    x_1 + x_2 > 0
